# Will it be delayed?

Everyone who has flown has experienced a delayed or cancelled flight. Both airlines and airports would like to improve their on-time performance and predict when a flight will be delayed or cancelled several days in advance. You are being hired to build a model that can predict if a flight will be delayed. To learn more, you must schedule a meeting with your client (me). To schedule an appointment with your client, send an event request through Google Calendar for a 15 minute meeting. Both you and your project partner must attend the meeting. Come prepared with questions to ask your client. Remember that your client is not a data scientist and you will need to explain things in a way that is easy to understand. Make sure that your communications are efficient, thought out, and not redundant as your client might get frustrated and "fire" you (this only applies to getting information from your client, this does not necessary apply to asking for help with the actual project itself - you should continuously ask questions for getting help).

For this project you must go through most all steps in the checklist. You must write responses for all items as done in the homeworks, however sometimes the item will simply be "does not apply". Keep your progress and thoughts organized in this document and use formatting as appropriate (using markdown to add headers and sub-headers for each major part). Some changes to the checklist:

* Do not do the final part (launching the product).
* Your presentation will be done as information written in this document in a dedicated section (no slides or anything like that). It should include high-level summary of your results (including what you learned about the data, the "accuracy" of your model, what features were important, etc). It should be written for your client, not your professor or teammates. It should include the best summary plots/graphics/data points.
* The models and hyperparameters you should consider during short-listing and fine-tuning will be released at a later time (dependent on how far we get over the next two weeks).
* Data retrieval must be automatic as part of the code (so it can easily be re-run and grab the latest data). Do not commit any data to the repository.
* Your submission must include a pickled final model along with this notebook.

In [1]:
#Imports
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.model_selection import train_test_split 

Get the Data
============

1. **List the data you need and how much you need:**
    - National flight data for 2023 and 2024
    - Weather data covering all of the same dates, preferably daily.
2. **Find and document where you can get that data:**
    - All of the weather data is available on the NOAA website. You must go through and make an order for each individual airport. The link is here: https://www.ncei.noaa.gov/cdo-web/ 
    - Flight data is from: https://www.transtats.bts.gov/tables.asp?QO_VQ=EFD&QO_anzr=Nv4yv0r 
3. **Get access authorizations**:
   - You must agree to the terms of use and make an order (which is free for digital use).
4. **Create a workspace**: This notebook.
5. **Get the data**: 
    - Download all of the CSV files from the websites mentioned above
6. **Convert the data to a format you can easily manipulate**:
   - The data is all in one parquet file.

In [2]:
data = pd.read_parquet('combined.parquet')
columns_to_keep = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate', 'OriginAirportID', 'Origin', 'OriginCityName', 'OriginStateName' ,'DestAirportID', 'Dest', 'DestCityName', 'DestStateName', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'ArrTime', 'ArrDelayMinutes', 'Cancelled', 'CancellationCode', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'AirTime', 'Flights', 'Distance']
data = data[columns_to_keep]

In [8]:
weather_df = pd.read_csv('3964079.csv')

In [7]:
print("Weather dataset columns:", weather_df.columns)
print(weather_df.head())

Weather dataset columns: Index(['STATION', 'NAME', 'DATE', 'AWND', 'PGTM', 'PRCP', 'SNOW', 'SNWD',
       'TAVG', 'TMAX', 'TMIN', 'WDF2', 'WDF5', 'WSF2', 'WSF5', 'WT01', 'WT02',
       'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10'],
      dtype='object')
       STATION                                NAME        DATE   AWND  PGTM  \
0  USW00012839  MIAMI INTERNATIONAL AIRPORT, FL US  2023-01-01   4.47   NaN   
1  USW00012839  MIAMI INTERNATIONAL AIRPORT, FL US  2023-01-02   7.16   NaN   
2  USW00012839  MIAMI INTERNATIONAL AIRPORT, FL US  2023-01-03  10.51   NaN   
3  USW00012839  MIAMI INTERNATIONAL AIRPORT, FL US  2023-01-04   8.50   NaN   
4  USW00012839  MIAMI INTERNATIONAL AIRPORT, FL US  2023-01-05   5.37   NaN   

   PRCP  SNOW  SNWD  TAVG  TMAX  ...  WT01  WT02  WT03  WT04  WT05  WT06  \
0   0.0   NaN   NaN    76  86.0  ...   1.0   NaN   NaN   NaN   NaN   NaN   
1   0.0   NaN   NaN    76  84.0  ...   NaN   NaN   NaN   NaN   NaN   NaN   
2   0.0   NaN   NaN    77

In [10]:
import pandas as pd

# Convert the 'DATE' column to datetime format. The "YYYY-MM-DD" format is automatically recognized.
weather_df['DATE'] = pd.to_datetime(weather_df['DATE'])

# Extract year, month, and day components to match the flight dataset keys
weather_df['Year'] = weather_df['DATE'].dt.year
weather_df['Month'] = weather_df['DATE'].dt.month
weather_df['DayofMonth'] = weather_df['DATE'].dt.day

# Rename 'STATION' to 'Origin' if the weather station represents the departure airport.
weather_df.rename(columns={'STATION': 'Origin'}, inplace=True)

# Display a preview of the processed weather dataset
print("Processed weather data:")
print(weather_df[['Origin', 'DATE', 'Year', 'Month', 'DayofMonth']].head())

# --- Assuming the flight dataset is already loaded in flight_df ---
# For example:
# flight_df = pd.read_csv('path_to_flight_data.csv')

# Confirm that the flight dataset contains the expected columns:
print("Flight dataset columns:")
print(data.columns)

# Merge the datasets on the keys: Year, Month, DayofMonth, and Origin.
# A left merge is used to keep all flight records even if corresponding weather data is missing.
combined_df = pd.merge(
    data,
    weather_df,
    on=['Year', 'Month', 'DayofMonth', 'Origin'],
    how='left'
)

# Preview the combined dataset
print("Combined dataset preview:")
print(combined_df.head())


Processed weather data:
        Origin       DATE  Year  Month  DayofMonth
0  USW00012839 2023-01-01  2023      1           1
1  USW00012839 2023-01-02  2023      1           2
2  USW00012839 2023-01-03  2023      1           3
3  USW00012839 2023-01-04  2023      1           4
4  USW00012839 2023-01-05  2023      1           5
Flight dataset columns:
Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'OriginAirportID', 'Origin', 'OriginCityName', 'OriginStateName',
       'DestAirportID', 'Dest', 'DestCityName', 'DestStateName', 'DepTime',
       'DepDelay', 'DepDelayMinutes', 'ArrTime', 'ArrDelayMinutes',
       'Cancelled', 'CancellationCode', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'AirTime', 'Flights',
       'Distance'],
      dtype='object')
Combined dataset preview:
   Year  Month  DayofMonth  DayOfWeek  FlightDate  OriginAirportID Origin  \
0  2023     12          30          6  2023-12-30            12339  

7. **Ensure sensitive information is deleted or protected**: This is public data
8. **Check the size and type of data (time series, geographical, …)**:

<mark>TODO</mark>: report your information below. At this point, since you don't want to look at the data too closely, this is a quick evaluation about the number of features and their data types (note: remember that just because all values for a feature are a number doesn't mean that feature is numerical), the number of samples (including possible missing data), and any special considerations about the features such as:

   1. Is it a time series: 
      - Yes

   2. Are any of the features unusable for the business problem? Or are some not available for the business problem when the model will be used?: 
      - Yes, so far all of the features are usable except for diverted flights, because the client doesn't want those accounted

   3. Which feature(s) will be used as the target/label for the business problem? (including which are required to derive the correct label)
      

   4. Should any of the features be stratified during the train/test split to avoid sampling biases?
   

Do not look at the data too closely at this point since you have not yet split off the testing set. Basically, enough looking at it to understand *how* to split the test set off. It is likely you will have to review the website where the data came from to be able to understand some of the features.

Frame the Problem and Look at the Big Picture
=============================================


1. **Define the objective in business terms:** 
2. **How will your solution be used?**
3. **What are the current solutions/workarounds (if any)?** 
4. **How should you frame this problem?** 
5. **How should performance be measured? Is the performance measure aligned with the business objective?** *Don't worry about this for now*
6. **What would be the minimum performance needed to reach the business objective?** *Don't worry about this for now*
7. **What are comparable problems? Can you reuse (personal or readily available) experience or tools?** 
8. **Is human expertise available?** Not immediately.
9. **How would you solve the problem manually?** 
10. **List the assumptions you (or others) have made so far. Verify assumptions if possible.** 

In [11]:
data.describe()

,Year,Month,DayofMonth,DayOfWeek,OriginAirportID,DestAirportID,DepTime,DepDelay,DepDelayMinutes,ArrTime,ArrDelayMinutes,Cancelled,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,AirTime,Flights,Distance
count,1.482571e+07,1.482571e+07,1.482571e+07,1.482571e+07,1.482571e+07,1.482571e+07,1.463667e+07,1.463630e+07,1.463630e+07,1.462553e+07,1.459227e+07,1.482571e+07,2.995616e+06,2.995616e+06,2.995616e+06,2.995616e+06,2.995616e+06,1.459227e+07,14825707.0,1.482571e+07
mean,2.023509e+03,6.586377e+00,1.577153e+01,3.983661e+00,1.265432e+04,1.265433e+04,1.332564e+03,1.235834e+01,1.565029e+01,1.460146e+03,1.557941e+01,1.327458e-02,2.489971e+01,4.090163e+00,1.300443e+01,1.314020e-01,2.884685e+01,1.118473e+02,1.0,8.066236e+02
std,4.999182e-01,3.403419e+00,8.781058e+00,2.007278e+00,1.526151e+03,1.526147e+03,5.077571e+02,5.612929e+01,5.507429e+01,5.441017e+02,5.468914e+01,1.144481e-01,7.658620e+01,3.370453e+01,3.139895e+01,3.370272e+00,6.458379e+01,6.986950e+01,0.0,5.922690e+02
min,2.023000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.013500e+04,1.013500e+04,1.000000e+00,-9.900000e+01,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,1.0,1.100000e+01
25%,2.023000e+03,4.000000e+00,8.000000e+00,2.000000e+00,1.129200e+04,1.129200e+04,9.120000e+02,-6.000000e+00,0.000000e+00,1.045000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.100000e+01,1.0,3.730000e+02
50%,2.024000e+03,7.000000e+00,1.600000e+01,4.000000e+00,1.288900e+04,1.288900e+04,1.325000e+03,-2.000000e+00,0.000000e+00,1.502000e+03,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,9.400000e+01,1.0,6.490000e+02
75%,2.024000e+03,1.000000e+01,2.300000e+01,6.000000e+00,1.402700e+04,1.402700e+04,1.746000e+03,9.000000e+00,9.000000e+00,1.916000e+03,9.000000e+00,0.000000e+00,2.200000e+01,0.000000e+00,1.700000e+01,0.000000e+00,3.300000e+01,1.410000e+02,1.0,1.045000e+03
max,2.024000e+03,1.200000e+01,3.100000e+01,7.000000e+00,1.686900e+04,1.686900e+04,2.400000e+03,5.764000e+03,5.764000e+03,2.400000e+03,5.780000e+03,1.000000e+00,5.764000e+03,2.419000e+03,2.700000e+03,1.460000e+03,3.581000e+03,1.338000e+03,1.0,5.095000e+03
